In [ ]:
from import_modules import *

with open('config.yaml', 'r') as file:
    cfg = yaml.safe_load(file)

In [ ]:
folder_path = r"data/"

sim = SIM(cfg["sim"]["N"], cfg["sim"]["timesteps"], cfg["sim"]["dt"], cfg["sim"]["alpha"], cfg["sim"]["beta"])

run = "no"

if run == "yes":
    print("No saved data, running simulation.")
    
    qs, ps = sim.integrate()
    # np.save("data_qs_2.npy", qs)
    # np.save("data_ps_2.npy", ps)

else:
    print("Found some saved data, skipping simulation.")
    
    qs = np.load(folder_path+r"data_qs_1.npy")
    ps = np.load(folder_path+r"data_ps_1.npy")

In [ ]:
ml = ML(window_size=cfg["model"]["window_size"], step_size=cfg["model"]["step_size"], train_size=cfg["model"]["train_size"], val_size=cfg["model"]["val_size"], test_size=cfg["model"]["test_size"])

X, y = ml.make_sequences(qs)

X_train, X_test, X_val, y_train, y_test, y_val = ml.make_split(X, y)

X_train = torch.Tensor(np.array(X_train))
X_test = torch.Tensor(np.array(X_test))
X_val = torch.Tensor(np.array(X_val))
y_train = torch.Tensor(np.array(y_train))
y_test = torch.Tensor(np.array(y_test))
y_val = torch.Tensor(np.array(y_val))

In [ ]:
print(len(X_train)//cfg["train"]["batch_size"])

In [ ]:
# Create model
model = Model(cfg["sim"]["N"]-2, cfg["model"]["hidden_units"], cfg["sim"]["N"]-2, cfg["model"]["dropout"], cfg["model"]["num_layers"])

In [ ]:
pytorch_total_params = sum(p.numel() for p in model.parameters())
print("Total number of parameters =", pytorch_total_params)

In [ ]:
# Create PyTorch datasets
train_dataset = TensorDataset(X_train, y_train)
val_dataset = TensorDataset(X_val, y_val)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=cfg["train"]["batch_size"], shuffle=False)
val_loader = DataLoader(val_dataset, batch_size=cfg["train"]["batch_size"], shuffle=False)

In [ ]:
# Define the loss function and optimizer
criterion = torch.nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=cfg["train"]["learn_rate"])

# Move the model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

In [ ]:
train_loss, val_loss, model = ml.gradient_descent(model, cfg["train"]["epochs"], train_loader, optimizer, device, val_loader, train_dataset, val_dataset, criterion, verbose=1)

In [ ]:
plt.plot(train_loss, color='tab:blue')
plt.plot(val_loss, color='tab:orange')
plt.grid(alpha=0.1)

In [ ]:
# Perform test predictions
test_pred = ml.predictions(model, device, X_test)

y_test = np.array(y_test)

In [ ]:
limit = 500

for particles in range(N-2):
    plt.plot(ml.postprocess(y_test)[:limit, particles], color="tab:blue")
    plt.plot(ml.postprocess(test_pred)[:limit, particles], color="tab:orange")

    # Set labels and title
    plt.xlabel('X')
    plt.ylabel('Y')
    # plt.ylim(-1.2, 1.2)

    # Show the plot
    plt.show()